# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [14]:
import os
os.chdir("C:/Users/Parolin/Documents/Projects/AEco")

aircraft_type = 'cargo'  #pax or cargo

input_path = './Data/mistral_whatif.xlsx'
input_sheet = 'Composite'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_mistral.xlsx'

iterations = 100000
chunks = 100000  #'auto' or more than 500

In [2]:
import os
os.chdir("C:/Users/Parolin/Documents/Projects/AEco")

aircraft_type = 'cargo'  #pax or cargo

input_path = './Data/mistral_whatif.xlsx'
input_sheet = 'Composite_fuel'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_mistral.xlsx'

iterations = 100000
chunks = 10000  #'auto' or more than 500

In [2]:
from Tools import *
from Model import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [15]:
del inputs, p, inventory

NameError: name 'inventory' is not defined

In [16]:
inputs = read_inputs(input_path, input_sheet)

In [17]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates PKM for the study.

In [25]:
p = pkm(aircraft_type, p)

<br></br>
Executing the UP reading and writing functions.

In [21]:
UP_dataframe = read_unit_processes(database_path)

In [22]:
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [26]:
inventory = LCI(name=input_sheet, type=aircraft_type, iterations=iterations, UP=UP, parameters=p)

In [27]:
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 100000)
Coordinates:
    Units           (Substances) object 'kg' 'm3' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconi...
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'urban air close to ground' ... 'non-...
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables:
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Sustaining      (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    LTO             (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    CCD             (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Maintenance     (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Airport         (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 100000), meta=np.ndarray>
Attributes:
    Name:     Composite

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [28]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

In [29]:
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [30]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [31]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconi...
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'urban air close to ground' ... 'non-...
Data variables:
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 1.0 0.0456 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.56 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    PMF             (Substances) float64 0.0 0.0 0.22 0.0 ... 0.0 0.0 0.0 0.0
    TET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    NLT             (Substances) float64 -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 -0.0
    OD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ME              (Substances) float64 0.0 0.0 0.039 0.0 ... 0.0 0.0 0.0 0.0
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [26]:
del composite

In [32]:
composite = LCIA.build(inventory, CF)

In [33]:
composite.mean(pathway="EP", by="sum")

<xarray.DataArray 'stack-f439f5d4ea70c78bf3ee4f52b4776553' (AOP: 3)>
dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

In [34]:
composite.MP

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 100000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables:
    Office          (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Sustaining      (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    LTO             (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    CCD             (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Maintenance     (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Airport         (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Fuel            (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(100000, 18), meta=np.ndarray>
Attributes:
    Name:     Composite

In [32]:
composite_fuel = LCIA.build(inventory, CF)

In [34]:
composite_fuel.mean(pathway="EP", by="sum")

<xarray.DataArray 'stack-89dbdc8cd1dfbe423a03f7548e67ecb9' (AOP: 3)>
dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [ ]:
composite.save(output_path, LCI=False)

In [22]:
#aeco.to_excel(output_path)

# What-if comparison

In [17]:
echo_path = '.\\Outputs\\Echo_outputs'  # LCA results file path
echo = LCIA.load(echo_path, chunks={}, LCI=False, CTV=False)

In [ ]:
echo.dist_compare(composite, pathway='EP', save=True)